# Assignment 1
- It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?
- Take this monstrosity as the DataFrame to use in the following puzzles:

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [23]:
import pandas as pd
import numpy as np
df= pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 'Budapest_PaRis', 'Brussels_londOn'], 'FlightNumber': [10045, np.nan, 10065, np.nan, 10085], 'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]], 'Airline': ['KLM(!)', ' (12)', '(British Airways. )', '12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],(12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


- 1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [24]:
df['FlightNumberNA']= df['FlightNumber'].isna() # create a flag column to keep track of which FlightNumber is missing
df['FlightNumber']= df['FlightNumber'].fillna(method= 'ffill') # fill NA values using forward fill 
df['FlightNumber']= df['FlightNumber']+np.where(df['FlightNumberNA'].eq(True), 10, 0) # add 10 where FlightNumberNA is True
df['FlightNumber']= df['FlightNumber'].astype(int)# make the column datatype as integer
df.drop(['FlightNumberNA'], axis= 1, inplace= True)
df # check the df FlightNumber column

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],(12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [25]:
df.info() # check the df datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   From_To       5 non-null      object
 1   FlightNumber  5 non-null      int64 
 2   RecentDelays  5 non-null      object
 3   Airline       5 non-null      object
dtypes: int64(1), object(3)
memory usage: 288.0+ bytes


- 2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [26]:
def get_Source(row):
    return row.split('_')[0]

def get_Destination(row):
    return row.split('_')[1]

temp= pd.DataFrame(columns= ['Source', 'Destination']) # create new temporary DataFrame
temp['Source']= df['From_To'].apply(get_Source) # assign the first location as Source
temp['Destination']= df['From_To'].apply(get_Destination) # assign the second location as Destination
temp # check the new temporary DataFrame

,Source,Destination
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


- 3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [27]:
def standardize(row):
    return row.capitalize() # Converts only first letter is uppercase, everything else to lowercase
    
for i in temp.columns:
    temp[i]= temp[i].apply(standardize) # apply the function row wise for each column
    
temp # check the standardized columns in temporary DataFrame

,Source,Destination
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


- 4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [28]:
df.drop(['From_To'], axis= 1, inplace= True) #Delete the From_To column
df= pd.concat([df, temp], axis= 1) # attach the temporary DataFrame
del temp # release memory held by temporary DataFrame

In [29]:
df

,FlightNumber,RecentDelays,Airline,Source,Destination
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],(12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


- 5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each
second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [30]:
delays= pd.DataFrame(columns= ['Delays'])
delays['Delays']= df['RecentDelays'].copy() # Expand the Series of lists into a DataFrame named delays
delays

,Delays
0,"[23, 47]"
1,[]
2,"[24, 43, 87]"
3,[13]
4,"[67, 32]"


In [31]:
def get_max_len(col): # get the maximum number of values of items in the lists so we can know how many new columns to create
    maxx= 0
    for i in col:
        maxx= len(i) if len(i)>maxx else maxx
    return maxx

max_len_delays= get_max_len(delays['Delays'])
for i in range(1, max_len_delays+1):
    col_name= 'delay_'+str(i)
    delays[col_name]= np.NaN  # create new columns & fill them with NULL value
    
for i, r in delays.iterrows():
    c= 0
    for j in r['Delays']: # fill in the existing values accordingly at the correct positions
        c= c+1
        col_name= 'delay_'+str(c)
        delays.loc[i, col_name]= j 
        
delays

,Delays,delay_1,delay_2,delay_3
0,"[23, 47]",23.0,47.0,NaN
1,[],NaN,NaN,NaN
2,"[24, 43, 87]",24.0,43.0,87.0
3,[13],13.0,NaN,NaN
4,"[67, 32]",67.0,32.0,NaN


In [32]:
# replacing the unwanted RecentDelays column in df with delays
df= pd.concat([df, delays], axis= 1)
df.drop(['RecentDelays'], axis= 1, inplace= True)
df

,FlightNumber,Airline,Source,Destination,Delays,delay_1,delay_2,delay_3
0,10045,KLM(!),London,Paris,"[23, 47]",23.0,47.0,NaN
1,10055,(12),Madrid,Milan,[],NaN,NaN,NaN
2,10065,(British Airways. ),London,Stockholm,"[24, 43, 87]",24.0,43.0,87.0
3,10075,12. Air France,Budapest,Paris,[13],13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels,London,"[67, 32]",67.0,32.0,NaN
